In [1]:
import json
import os,sys
import pandas as pd
import numpy as np
import xarray
from datetime import datetime
from shapely.geometry import Point, Polygon
from ast import literal_eval

In [2]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)


In [3]:

ff = pd.read_json('/root/data/rrr/integrated_weather_dataset/data/raw/Flash_Flood/ffw_cp.json')
print(ff)

                                                                                                                                                                                                                                                                                                                                                                                                                                  FFWs
0                                                                                                                                                                                                                           {'begin': '2005-02-21T21:50:00Z', 'end': '2005-02-22T00:45:00Z', 'vertices': [{'lat': 35.66, 'lon': 120.11}, {'lat': 35.69, 'lon': 119.78}, {'lat': 36.75, 'lon': 120.19}, {'lat': 36.66, 'lon': 120.66}]}
1                                                                                                                                                         

In [4]:
ff.dtypes

FFWs    object
dtype: object

In [5]:
print(ff.iloc[0])

FFWs    {'begin': '2005-02-21T21:50:00Z', 'end': '2005-02-22T00:45:00Z', 'vertices': [{'lat': 35.66, 'lon': 120.11}, {'lat': 35.69, 'lon': 119.78}, {'lat': 36.75, 'lon': 120.19}, {'lat': 36.66, 'lon': 120.66}]}
Name: 0, dtype: object


In [6]:
MIN_LON = 120
MAX_LON = 115
MIN_LAT = 31.5
MAX_LAT = 38
start_date = '2005-01-01T00:00:00.000000000'
end_date = '2005-01-31T00:00:00.000000000'

In [7]:
timestamps = pd.date_range(start=start_date, end=end_date, freq='5min')
lats = np.arange(MIN_LAT, MAX_LAT, 0.5)
lons = np.arange(MAX_LON, MIN_LON, 0.625)


ffw_periods = []
ffw_polygons = []
for ffw in ff['FFWs']:
    start_time = pd.to_datetime(ffw['begin'], utc=False)
    end_time = pd.to_datetime(ffw['end'], utc=False)
    vertices = [(v['lon'], v['lat']) for v in ffw['vertices']]
    ffw_periods.append((start_time, end_time))
    ffw_polygons.append(Polygon(vertices))

In [8]:
ffw_periods = [(ts1.tz_localize(None), ts2.tz_localize(None)) for ts1, ts2 in ffw_periods]


In [ ]:
grid_points = []
for t in timestamps:
    is_ffw_time = any(start <= t <= end for start, end in ffw_periods)
    
    if is_ffw_time: 
        for lat in lats:
            for lon in lons:
                point = Point(lon, lat)
                in_polygon = any(poly.contains(point) for poly in ffw_polygons)
                
                if in_polygon: 
                    grid_points.append({
                        'timestamp': t,
                        'latitude': lat,
                        'longitude': lon,
                        'label': 1
                    })


In [ ]:
grid_points

In [ ]:
result_df = pd.DataFrame(grid_points)


In [ ]:
result_df